# Reading and Downloading DEM and LPC Products

The `coincident.io.xarray` and `coincident.io.download` modules support the option to load DEMs into memory via odc-stac or download DEMs into local directories. 

There is also support for Lidar Point Cloud (LPC) spatial filtering for aerial lidar catalogs, where the user can return a GeoDataFrame with the respective .laz tile filename, download url, and geometry (epsg 4326) for each tile intersecting an input aoi. These laz files can then be downloaded locally with `coincident.io.download.download_files()`

There is specific support for USGS 3DEP EPT readers where the user can return a PDAL pipeline configured with the EPT URL, the AOI's bounds, and polygon WKT, all in the EPT's spatial reference system.

```{note}
Coincident *does not support the processing* of lidar point cloud products. Please see the [lidar_tools](https://github.com/uw-cryo/lidar_tools) repository for information on processing the returned GeoDataFrame with lidar point cloud products.
```

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box

## 3DEP and NEON overlapping Flights

```{note}
For all of these functions, you will need identification metadata from the `coincident.search.search` functions for each respective catalog
```

### Search

In [ ]:
workunit = "CO_CentralEasternPlains_1_2020"
df_wesm = coincident.search.wesm.read_wesm_csv()
gf_usgs = coincident.search.wesm.load_by_fid(
    df_wesm[df_wesm.workunit == workunit].index
)

gf_usgs

In [ ]:
# We will examine the 'sourcedem' 1m
# NOTE: it's important to note the *vertical* CRS of the data
gf_usgs.iloc[0]

Now, we'll explore an overlapping NEON flight

In [ ]:
gf_neon = coincident.search.search(
    dataset="neon", intersects=gf_usgs, datetime=["2020"]
)

In [ ]:
gf_neon.head()

In [ ]:
m = gf_usgs.explore(color="black")
gf_neon.explore(m=m, column="id", popups=True)

### Subset data

We will evaluate small subset of this overlap for deomstrative purposes.

Let's subset based on some contextual LULC data

In [ ]:
gf_wc = coincident.search.search(
    dataset="worldcover",
    intersects=gf_neon,
    datetime=["2020"],
)

In [ ]:
dswc = coincident.io.xarray.to_dataset(
    gf_wc,
    bands=["map"],
    aoi=gf_neon,
    mask=True,
    resolution=0.00027,  # ~30m
)
dswc = dswc.rename(map="landcover")
dswc = dswc.compute()

In [ ]:
# arbitrary bbox that will be our subset area (all cropland)
bbox_geometry = box(-102.505, 39.675, -102.49, 39.685)
aoi = gpd.GeoDataFrame(geometry=[bbox_geometry], crs="EPSG:4326")

In [ ]:
ax = coincident.plot.plot_esa_worldcover(dswc.landcover)
aoi.plot(ax=ax, facecolor="none", edgecolor="black", linestyle="--", linewidth=2)
from matplotlib.lines import Line2D

custom_line = Line2D([0], [0], color="black", linestyle="--", lw=2)
ax.legend([custom_line], ["Area of Interest"], loc="upper right", fontsize=10)
ax.set_title("ESA WorldCover");

Actually read in the DEMs

In [ ]:
datetime_str = gf_neon.end_datetime.dt.strftime("%Y-%m-%d").item()
site_id = gf_neon.id.item()
datetime_str, site_id

In [ ]:
%%time
da_neon_dem = coincident.io.xarray.load_neon_dem(
    aoi, datetime_str=datetime_str, site_id=site_id, product="dsm"
)

In [ ]:
da_neon_dem

In [ ]:
usgs_project = gf_usgs["project"].item()
usgs_project

In [ ]:
%%time
da_usgs_dem = coincident.io.xarray.load_usgs_dem(aoi, usgs_project)

In [ ]:
da_usgs_dem

In [ ]:
da_usgs_dem.coarsen(x=5, y=5, boundary="trim").mean().plot.imshow();

### Download

```{note}
`coincident.io.download.download_neon_dem` needs the NEON site's start_datetime OR end_datetime to work
```

In [ ]:
gf_neon

In [ ]:
local_output_dir = "/tmp"

In [ ]:
coincident.io.download.download_neon_dem(
    aoi=aoi,
    datetime_str=gf_neon.end_datetime.dt.strftime("%Y-%m-%d").item(),
    site_id=gf_neon.id.item(),
    product="dsm",
    output_dir=local_output_dir,
)

In [ ]:
# USGS_1M_13_x71y440_CO_CentralEasternPlains_2020_D20.tif:  236MB
coincident.io.download.download_usgs_dem(
    aoi=aoi,
    project=usgs_project,
    output_dir=local_output_dir,
    save_parquet=True,  # save a STAC-like geoparquet of the tiles you download
)

Finally, you can grab the LPC tile metadata. For USGS 3DEP data, you can also return a PDAL pipeline based on the available EPT data. This PDAL pipeline will be returned as a JSON file where the user can add their own custom parameters (additional filters, writers, etc.) to this pipeline dictionary before executing it with PDAL.

```{note}
`coincident.io.download.fetch_lpc_tiles` needs the NEON site's end_datetime OR start_datetime to work if you are in fact accessing NEON data
```

In [ ]:
%%time
gf_neon_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=aoi,
    dataset_id=gf_neon.id.item(),
    provider="NEON",
    datetime_str=gf_neon.end_datetime.dt.strftime("%Y-%m-%d").item(),
)

In [ ]:
gf_neon_lpc_tiles.head()

In [ ]:
%%time
gf_usgs_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=aoi, dataset_id=usgs_project, provider="USGS"
)

In [ ]:
gf_usgs_lpc_tiles.head()

In [ ]:
m = gf_usgs_lpc_tiles.explore(color="black")
gf_neon_lpc_tiles.explore(m=m)

Now, we can download the laz files

In [ ]:
coincident.io.download.download_files(
    gf_neon_lpc_tiles["url"], output_dir=local_output_dir
)

In [ ]:
pdal_pipeline = coincident.io.download.build_usgs_ept_pipeline(
    aoi, workunit=gf_usgs.workunit.item(), output_dir=local_output_dir
)

In [ ]:
pdal_pipeline